In [8]:
import duckdb
from datasets import load_dataset

/Users/felix/MSE/03_projects/MT/06_rag/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
DB = '../results/2026-01-20_00-46/kilt_wiki.duckdb'
WIKI_ALL = '../data/wikipedia_kilt.duckdb'

In [1]:
# TODO: Collect queries from documents outside of the 10_000 index

In [3]:
con = duckdb.connect(DB)
source = duckdb.connect(WIKI_ALL)

In [4]:
con.execute("SELECT * FROM paragraph").df()

,wikipedia_id,wikipedia_title,global_id,index,text
0,324,Academy Awards,1,165,BULLET::::- Official Academy Awards Database (...
1,339,Ayn Rand,2,101,"BULLET::::- ""Writings of Ayn Rand"" – from C-SP..."
2,307,Abraham Lincoln,3,222,BULLET::::- Digitized items in the Alfred Whit...
3,358,Algeria,4,197,BULLET::::- EU Neighbourhood Info Centre: Alge...
4,20134,Minnesota Vikings,5,262,BULLET::::- List of Minnesota Vikings broadcas...
...,...,...,...,...,...
216884,5058739,Sweden,216885,5,Sweden is a constitutional monarchy and a parl...
216885,5058739,Sweden,216886,4,Germanic peoples have inhabited Sweden since p...
216886,5058739,Sweden,216887,3,Sweden is part of the geographical area of Fen...
216887,5058739,Sweden,216888,2,"Sweden ( ), formal name: the Kingdom of Sweden..."


In [5]:
page_ids = con.execute("SELECT wikipedia_id FROM wiki").df()
page_ids = page_ids.values.flatten().tolist()

In [7]:
unknown_pages = source.execute(f"SELECT * FROM wiki WHERE wikipedia_id NOT IN ({','.join(['?']*len(page_ids))}) LIMIT 100", page_ids).df()

In [9]:
allowed = ['SUPPORTS', 'REFUTES']

kilt_fever = load_dataset("kilt_tasks", name="fever")
train_clean = kilt_fever['train'].filter(lambda row: (row['output'][0]['answer'] in allowed and len(row['output'][0]['provenance']) > 0))
train_clean

Dataset({
    features: ['id', 'input', 'meta', 'output'],
    num_rows: 67076
})

In [13]:
unseen_pages = train_clean.filter(lambda row: row['output'][0]['provenance'][0]['wikipedia_id'] not in page_ids)

Filter: 100%|██████████| 67076/67076 [00:03<00:00, 17371.24 examples/s]


In [ ]:
unseen_pages[0]

Dataset({
    features: ['id', 'input', 'meta', 'output'],
    num_rows: 1559
})

In [20]:
unseen_pages.to_json("../results/validation/unseen.jsonl")

Creating json from Arrow format: 100%|██████████| 2/2 [00:00<00:00, 18.34ba/s]


1169059